In [1]:
import tensorflow as tf
from core.yolov4 import YOLOv4, decode, filter_boxes
import core.utils as utils
import cv2
import numpy as np

In [2]:
weights = 'models/yolov4-default/yolov4.weights' #path to weights file
anchors = 'models/yolov4-default/anchors.txt'
classes = 'models/yolov4-default/classes.txt'
output1 = './checkpoints/yolov4.h5' #path to output
output2 = './checkpoints/yolov4' #path to output

input_size = 416 #define input size of export model
score_thres = 0.2 #define score threshold'

In [3]:
ANCHORS, NUM_CLASS = utils.load_config(anchors, classes)

input_layer = tf.keras.layers.Input([input_size, input_size, 3])
feature_maps = YOLOv4(input_layer, NUM_CLASS)
bbox_tensors = []
prob_tensors = []

for i, fm in enumerate(feature_maps):
    if i == 0:
        output_tensors = decode(fm, input_size // 8, ANCHORS, NUM_CLASS, i)
    elif i == 1:
        output_tensors = decode(fm, input_size // 16, ANCHORS, NUM_CLASS, i)
    else:
        output_tensors = decode(fm, input_size // 32, ANCHORS, NUM_CLASS, i)
    bbox_tensors.append(output_tensors[0])
    prob_tensors.append(output_tensors[1])
pred_bbox = tf.concat(bbox_tensors, axis=1)
pred_prob = tf.concat(prob_tensors, axis=1)

boxes, pred_conf = filter_boxes(pred_bbox, pred_prob, score_threshold=score_thres, input_shape=tf.constant([input_size, input_size]))
pred = tf.concat([boxes, pred_conf], axis=-1)

model = tf.keras.Model(input_layer, pred)
utils.load_weights(model, weights)

#model.summary()
#model.save(output2)

In [8]:

original_image = cv2.imread('data/car.jpg')
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

image_data = cv2.resize(original_image, (input_size, input_size))
image_data = image_data / 255.

image_data = np.array([image_data]).astype(np.float32)

result = model.predict(image_data)
print(result)

1/1 [==============================] - 0s 34ms/step
[]


In [5]:
result.shape

(1, 32, 84)